In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
import pyarrow
from datetime import datetime

## 1. 数据源与 Azure 连接（未来迁移到 azure_connector.py）
### TODO：后续提取为 src/azure_connector.py -> AzureConnector 类

In [2]:
import sys
sys.path.append('../src')

from azure_connector import AzureConnector

In [ ]:
account_name = 'beadbatch'
account_key = 'SWKKxmypIa/oz59CsQo8xHectQEv+jEJUKuG17qDzBFAJyWqDsuAhMtLymDaeDw0iu1BCbLof5lK+AStGm/VnQ=='
container_name = 'batch'
file_name = 'Combined_Flights_2022.parquet'

azure = AzureConnector(account_name, account_key)
df = azure.read_parquet(container_name, file_name)

print("成功读取 Parquet 文件：")
display(df.head())

✅ 成功读取 Parquet 文件：


,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",GJT,DEN,False,False,1133,1123.0,0.0,-10.0,...,1140.0,1220.0,8.0,1245,-17.0,0.0,-2.0,1200-1259,1,0
1,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",HRL,IAH,False,False,732,728.0,0.0,-4.0,...,744.0,839.0,9.0,849,-1.0,0.0,-1.0,0800-0859,2,0
2,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1529,1514.0,0.0,-15.0,...,1535.0,1622.0,14.0,1639,-3.0,0.0,-1.0,1600-1659,2,0
3,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,GPT,False,False,1435,1430.0,0.0,-5.0,...,1446.0,1543.0,4.0,1605,-18.0,0.0,-2.0,1600-1659,2,0
4,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1135,1135.0,0.0,0.0,...,1154.0,1243.0,8.0,1245,6.0,0.0,0.0,1200-1259,2,0


In [ ]:
# 统计数据类型
column_type_dict = {col: str(dtype) for col, dtype in df.dtypes.items()}
print(column_type_dict)

{'FlightDate': 'datetime64[ns]', 'Airline': 'object', 'Origin': 'object', 'Dest': 'object', 'Cancelled': 'bool', 'Diverted': 'bool', 'CRSDepTime': 'int64', 'DepTime': 'float64', 'DepDelayMinutes': 'float64', 'DepDelay': 'float64', 'ArrTime': 'float64', 'ArrDelayMinutes': 'float64', 'AirTime': 'float64', 'CRSElapsedTime': 'float64', 'ActualElapsedTime': 'float64', 'Distance': 'float64', 'Year': 'int64', 'Quarter': 'int64', 'Month': 'int64', 'DayofMonth': 'int64', 'DayOfWeek': 'int64', 'Marketing_Airline_Network': 'object', 'Operated_or_Branded_Code_Share_Partners': 'object', 'DOT_ID_Marketing_Airline': 'int64', 'IATA_Code_Marketing_Airline': 'object', 'Flight_Number_Marketing_Airline': 'int64', 'Operating_Airline': 'object', 'DOT_ID_Operating_Airline': 'int64', 'IATA_Code_Operating_Airline': 'object', 'Tail_Number': 'object', 'Flight_Number_Operating_Airline': 'int64', 'OriginAirportID': 'int64', 'OriginAirportSeqID': 'int64', 'OriginCityMarketID': 'int64', 'OriginCityName': 'object', '

## 2. 数据预览与初步探索（EDA）

## 3. 数据清洗 & 特征工程（未来迁移到 src/data_processor.py）

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [12]:
# 选取特征（排除 ArrDelayMinutes 自身）
features = [
    'Airline', 'Origin', 'Dest', 'CRSDepTime', 'DepDelay', 'Distance',
    'Month', 'DayofMonth', 'DayOfWeek', 'DepDel15', 'TaxiOut', 'CRSArrTime'
]

target = 'ArrDelayMinutes'

# 处理时间特征（可选，把 CRSDepTime 转成小时）
df['CRSDepHour'] = (df['CRSDepTime'] // 100).astype('int')
df['CRSArrHour'] = (df['CRSArrTime'] // 100).astype('int')
features.extend(['CRSDepHour', 'CRSArrHour'])

# 缺失值简单处理
df[features] = df[features].fillna(0)
df[target] = df[target].fillna(df[target].mean())
df['DepDelay'] = df['DepDelay'].fillna(df['DepDelay'].mean())
df['TaxiOut'] = df['TaxiOut'].fillna(0)

# 类别特征编码
for col in ['Airline', 'Origin', 'Dest']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))

## 4. 模型训练（未来迁移到 src/modeling.py）

In [13]:
# 准备建模数据
X = df[features]
y = df[target]

# 标准化（可选，给回归模型提升表现）
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 划分训练测试集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 线性回归建模
model = LinearRegression()
model.fit(X_train, y_train)

# 预测与评估
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"预测完成，RMSE: {rmse:.2f}, R2 Score: {r2:.4f}")

预测完成，RMSE: 10.04, R2 Score: 0.9615
